# Preprocessing

### Read the Enron datasets

In [2]:
import pandas as pd

df = pd.read_csv("./emails.csv")
df.head()

FileNotFoundError: [Errno 2] No such file or directory: './emails.csv'

In [ ]:
import email
messages = df['message'].values.tolist()
# emails = [  for messsage in messages: email.message_from_string(message)]
# emails = [] 
# for message in messages:
#     e = email.message_from_string(message)
#     emails.append(e)

In [ ]:
def body(messages):
    column = []
    for message in messages:
        e = email.message_from_string(message)
        msg = e.get_payload()
        # -----Original Message----- 以降を切り捨てる
        list1 = msg.split("-----Original Message-----")
        if len(list1) >0:
            msg = list1[0]
        
        column.append(msg)
    return column

In [ ]:
def author(messages):
    column = []
    for message in messages:
        e = email.message_from_string(message)
        msg = e.get('From')
        ## If you use 'X-From'
        # if msg is None:
        #     msg = "UNKNOWN"
        # list2 = msg.split('<')
        # if len(list2) >0:
        #     msg = list2[0]
        
        column.append(msg)
    return column

In [ ]:
new_df = pd.DataFrame(
    data={
        'author': author(messages),
        'body': body(messages)  
        }

)

In [ ]:
new_df

,author,body
0,phillip.allen@enron.com,Here is our forecast\n\n
1,phillip.allen@enron.com,Traveling to have a business meeting takes the...
2,phillip.allen@enron.com,test successful. way to go!!!
3,phillip.allen@enron.com,"Randy,\n\n Can you send me a schedule of the s..."
4,phillip.allen@enron.com,Let's shoot for Tuesday at 11:45.
...,...,...
517396,john.zufferli@enron.com,This is a trade with OIL-SPEC-HEDGE-NG (John L...
517397,john.zufferli@enron.com,Some of my position is with the Alberta Term b...
517398,john.zufferli@enron.com,2\n\n
517399,john.zufferli@enron.com,Analyst\t\t\t\t\tRank\n\nStephane Brodeur\t\t\...


## Remove the data incruding "Forwarded by"

In [ ]:
import numpy as np
new_df = new_df[
            (new_df['body'].str.contains('Forwarded by') == False )
                # & ( new_df['body'] != None)
                # & (new_df['body'] != "")
                & (new_df['body'].isnull() == False)
                # & (new_df['body'] != np.nan)

                ]

In [ ]:
new_df

,author,body
0,phillip.allen@enron.com,Here is our forecast\n\n
1,phillip.allen@enron.com,Traveling to have a business meeting takes the...
2,phillip.allen@enron.com,test successful. way to go!!!
3,phillip.allen@enron.com,"Randy,\n\n Can you send me a schedule of the s..."
4,phillip.allen@enron.com,Let's shoot for Tuesday at 11:45.
...,...,...
517396,john.zufferli@enron.com,This is a trade with OIL-SPEC-HEDGE-NG (John L...
517397,john.zufferli@enron.com,Some of my position is with the Alberta Term b...
517398,john.zufferli@enron.com,2\n\n
517399,john.zufferli@enron.com,Analyst\t\t\t\t\tRank\n\nStephane Brodeur\t\t\...


In [ ]:
authors = {} #Data type as Dictionary 

for author in new_df['author']:
    if author in authors:
        authors[author] += 1
    else:
        authors[author] = 1

sorted_authors = sorted(authors.items(), key=lambda x: x[1], reverse=True)

# Remake the func words < 1000 

In [ ]:
selected_authors = []
i = 0
num_author =5
word_upper_limit = 500
for name, count in sorted_authors:
    if i > num_author:
        break
    if count <= word_upper_limit:
        selected_authors.append(name)
        i = i + 1

In [ ]:
for author in selected_authors:
    print(f'{author}: {authors[author]}')

chris.stokley@enron.com: 493
v.weldon@enron.com: 491
j..kean@enron.com: 483
andy.zipper@enron.com: 468
hunter.shively@enron.com: 464
ben.jacoby@enron.com: 458


In [ ]:
reduced_df = new_df[new_df['author'] == selected_authors[0]]
for name in selected_authors[1:]:
    reduced_df = pd.concat([reduced_df, new_df[new_df['author'] == name]], join='inner')


# Hello world!!

In [ ]:
reduced_df

,author,body
157494,chris.stokley@enron.com,Please view the summary tab on each worksheet ...
159437,chris.stokley@enron.com,Please view the summary tab on each worksheet ...
160619,chris.stokley@enron.com,Please view the summary tab on each worksheet ...
282113,chris.stokley@enron.com,Please stop by this morning for cake and cooki...
283054,chris.stokley@enron.com,Please stop by this morning for cake and cooki...
...,...,...
439021,ben.jacoby@enron.com,I talked to Bill Butler after this came out. B...
439023,ben.jacoby@enron.com,"Gentlemen:\n\nFurther to our conversation, ple..."
439026,ben.jacoby@enron.com,I talked to Bill Butler after this came out. B...
440054,ben.jacoby@enron.com,Charles:\n\nIt was good to catch up with you t...


In [ ]:
reduced_df.to_csv("preprocessed_enron.csv", index=False)

# 前処理終了!!!!!!

# ここから、まずTF-IDF 
